In [ ]:
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from metpy.plots import USCOUNTIES
import cartopy.crs as ccrs 
import cartopy.feature as cfeature
import numpy as np
from pathlib import Path
from typing import Union
import metpy
import os
import scipy.ndimage as ndimage
import pandas as pd
from metpy.plots import StationPlot
import metpy.calc as mpcalc
from metpy.units import units

In [ ]:
## A couple functions are from Tim Supinie's gridradpy repository on GitHub (https://github.com/tsupinie/gridradpy/tree/main) with some modifications ##

_index_variables = ['Reflectivity', 'wReflectivity', 'SpectrumWidth', 'wSpectrumWidth', 'AzShear', 'wAzShear', 
                    'Divergence', 'wDivergence', 'DifferentialReflectivity', 'wDifferentialReflectivity',
                    'DifferentialPhase', 'wDifferentialPhase', 'CorrelationCoefficient', 'wCorrelationCoefficient']

def read_file(infile: Union[str, Path]) -> xr.Dataset:
    ds = xr.open_dataset(infile)

    nlon = ds.dims['Longitude']
    nlat = ds.dims['Latitude']
    nalt = ds.dims['Altitude']

    index = ds['index'].values

    da_dict = {}
    for var in _index_variables:
        if var not in ds.variables:
            continue

        # Create arrays to store binned values for reflectivity at horizontal polarization
        values    = np.zeros(nlon * nlat * nalt, dtype=np.float32)
        values[:] = np.nan

        # Add values to arrays
        values[index[:]]  =  ds[var].values[:]
        da = xr.DataArray(data=values.reshape((nalt, nlat, nlon)), coords=ds['Nradobs'].coords, 
                          dims=ds['Nradobs'].dims, name=ds[var].name, attrs=ds[var].attrs)

        da_dict[var] = da

    ds = ds.assign(**da_dict)

    return ds.drop_vars('index')

# GridRad filter routine
def filter(ds: xr.Dataset, wthresh=1.5, freq_thresh=0.6, Z_H_thresh=15.0, nobs_thresh=2):
    """
    wthresh:        Bin weight threshold for filtering by year (dimensionless)
    freq_thresh:    Echo frequency threshold (dimensionless)
    Z_H_thresh:     Reflectivity threshold (dBZ)
    nobs_thresh:    Number of observations threshold
    """

    has_data = ds['Nradobs'] > 0
    echo_frequency = (ds['Nradecho'] / ds['Nradobs']).where(has_data, 0.)

    # Find observations with low weight
    mask = ~(((ds['wReflectivity'] < wthresh) & (ds['Reflectivity'] < Z_H_thresh)) |
             ((echo_frequency < freq_thresh) & (ds['Nradobs'] > nobs_thresh)))
    
    # Remove low confidence observations
    if has_data.any():
        da_dict = {}
        for var in _index_variables:
            if var.startswith('w') or var not in ds.variables:
                continue
            
            da_dict[var] = ds[var].where(mask)

        ds = ds.assign(**da_dict)
    
    # Return filtered data0
    return ds

# Gridrad clutter filter routine
def remove_clutter(ds: xr.Dataset, skip_weak_ll_echo=False, areal_coverage_thresh=0.32) -> xr.Dataset:
    """
    areal_coverage_thresh:  Fractional areal coverage threshold for speckle identification
    """
    
    da = ds['Reflectivity']
    clutter = xr.DataArray(data=np.zeros_like(da.values, dtype=bool), coords=da.coords, dims=da.dims)

    # Light pass at a correlation coefficient decluttering approach first
    if 'DifferentialReflectivity' in ds.variables:
        cc_clutter = (((ds['Reflectivity'] < 40.) & (ds['CorrelationCoefficient'] < 0.9)) | 
                      ((ds['Reflectivity'] < 25.) & (ds['CorrelationCoefficient'] < 0.95) & (ds['Altitude'] > 10.)))

			        
    # First pass at removing speckles
    # TAS: This is slightly different than the original. It fills the boundaries with the nearest neighbor instead of 
    #   wrapping around to the other side of the domain
    has_refl_data = ~ds['Reflectivity'].where(~clutter).isnull()
    cover = (has_refl_data.rolling(Longitude=5, Latitude=5, center=True).mean()
                          .ffill(dim='Longitude').bfill(dim='Longitude')
                          .ffill(dim='Latitude').bfill(dim='Latitude'))
    speckle = cover <= areal_coverage_thresh
    clutter = clutter | speckle

    # Attempts to mitigate ground clutter and biological scatterers
    if not skip_weak_ll_echo:
        # Find weak low-level echoes
        weakref_clutter = ((ds['Reflectivity'].where(~clutter) < 10.) & (ds['Altitude'] <= 4.))
        clutter = clutter | weakref_clutter

        # Second check for weak, low-level echo
        refl_da = ds['Reflectivity'].where(~clutter)
        refl_max = refl_da.max(dim='Altitude')
        echo0_min  = ((refl_da >  0.) * ds['Altitude']).min(dim='Altitude')
        echo0_max  = ((refl_da >  0.) * ds['Altitude']).max(dim='Altitude')
        echo5_max  = ((refl_da >  5.) * ds['Altitude']).max(dim='Altitude')
        echo15_max = ((refl_da > 15.) * ds['Altitude']).max(dim='Altitude')
        
        # Find weak and/or shallow echo
        col_mask = (((refl_max   <  20.) & (echo0_max  <= 4.) & (echo0_min  <= 3.)) |
                    ((refl_max   <  10.) & (echo0_max  <= 5.) & (echo0_min  <= 3.)) |
                    ((echo5_max  <=  5.) & (echo5_max  >  0.) & (echo15_max <= 3.)) |
                    ((echo15_max <   2.) & (echo15_max >  0.)))

        clutter = clutter | col_mask

    # Find clutter below convective anvils
    # TAS: The original code cuts off the topmost horizontal slice of the above- and below-4-km layers. I'm guessing 
    #   that's not correct.
    alt_cutoff = 4.
    has_refl_data = ~ds['Reflectivity'].where(~clutter).isnull()
    anvil_clutter = ((has_refl_data.sel(Altitude=alt_cutoff) == False) & 
                     (has_refl_data.sel(Altitude=slice(None, alt_cutoff)).sum(dim='Altitude') > 0) &
                     (has_refl_data.sel(Altitude=slice(alt_cutoff, None)).sum(dim='Altitude') > 0) &
                     (ds['Altitude'] <= alt_cutoff))

    clutter = clutter | anvil_clutter
    
    # Second pass at removing speckles
    has_refl_data = ~ds['Reflectivity'].where(~clutter).isnull()
    cover = (has_refl_data.rolling(Longitude=5, Latitude=5, center=True).mean()
                          .ffill(dim='Longitude').bfill(dim='Longitude')
                          .ffill(dim='Latitude').bfill(dim='Latitude'))
    speckle = cover <= areal_coverage_thresh
    clutter = clutter | speckle

    # Remove the clutter from all variables
    da_dict = {}
    for var in _index_variables:
        if var.startswith('w') or var not in ds.variables:
            continue
        
        da_dict[var] = ds[var].where(~clutter)

    ds = ds.assign(**da_dict)
    
    return ds

def plot_image(df, ds):
    cmap = metpy.plots.ctables.registry.get_colortable('NWSReflectivity')
    ref = ds['Reflectivity'].sel(Altitude=3)
    lons = ds['Longitude']
    lats = ds['Latitude']
    u, v = mpcalc.wind_components(df['sknt'].values * units.knots, df['drct'].values * units.degrees)

    fig, ax = plt.subplots(figsize=(10, 6),subplot_kw={'projection': ccrs.PlateCarree()})

    ax.set_extent([-94.5, -83, 41, 49])
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=0.5)
    ax.add_feature(cfeature.STATES, linewidth=0.5)
    ax.add_feature(USCOUNTIES.with_scale('5m'), linewidth=0.25)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)

    stationplot = StationPlot(ax, df['lon'].values, df['lat'].values, transform=ccrs.PlateCarree(),
                          fontsize=10, color='dimgray')
    stationplot.plot_parameter('NW', df['tmpf'], color='red')
    stationplot.plot_parameter('SW', df['dwpf'], color='green')
    stationplot.plot_barb(u, v)

    raob_lon, raob_lat = [-88.11,-84.71,-93.56],[44.5,44.91,44.85]
    buoy_lon, buoy_lat = [-86.342, -86.411, -86.514, -87.758, -87.360,-87.134], [44.251, 45.344, 44.055, 44.794, 45.198,42.674]

    plt.scatter(raob_lon, raob_lat, c='black',s=150, zorder=2,label='RAOB Site', marker ="X")
    plt.scatter(buoy_lon, buoy_lat, c='black',s=125, zorder=2,label='Buoy', marker ="d")

    plt.pcolormesh(ds['Longitude'], ds['Latitude'], ref, cmap=cmap, vmin=0, vmax=75)
    plt.colorbar(shrink=0.85, label='dBZ')
    plt.title('{} {}z'.format(ref.name, ds['time'].dt.strftime('%Y-%m-%d %H:%M:%S').values[0]))
    ax.set_xlabel("X_axis_title")
    #plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    ax.gridlines(draw_labels=True, color='black',linestyle='--', alpha=0.35)
    plt.legend(loc='upper right') 
    plt.tight_layout()
    plt.savefig('FRMETR_4990_GridRad_{}_{}z'.format(ref.name, ds['time'].dt.strftime('%Y-%m-%d_%H-%M-%S').values[0]), dpi=450)
    plt.show()


def read_multiple_urls(urls):
    dfs = []
    for url in urls:
        df = pd.read_csv(url)
        dfs.append(df)
    concatenated_df = pd.concat(dfs, ignore_index=True)
    return concatenated_df

def preprocess_data(df):
    columns_to_keep = ['valid', 'station', 'lon', 'lat', 'tmpf', 'dwpf', 'drct', 'sknt']
    df = df[columns_to_keep]
    df = df.dropna(subset=['tmpf'])
    df.replace('M', np.nan, inplace=True)
    df = df.dropna(subset=['tmpf', 'dwpf'])
    
    # Convert columns to numeric
    df['tmpf'] = pd.to_numeric(df['tmpf'], errors='coerce')
    df['dwpf'] = pd.to_numeric(df['dwpf'], errors='coerce')
    df['sknt'] = pd.to_numeric(df['sknt'], errors='coerce')
    df['drct'] = pd.to_numeric(df['drct'], errors='coerce')
    df['time'] = pd.to_datetime(df['valid'])
    
    lat_min = 41.5
    lat_max = 48.5
    lon_min = -93.5
    lon_max = -83.5
    filtered_df = df[(df['lat'] >= lat_min) & (df['lat'] <= lat_max) &
                     (df['lon'] >= lon_min) & (df['lon'] <= lon_max)]
    
    return filtered_df

def obs(df, ds):
    single_time = ds.time.values[0] 
    time_threshold = pd.Timedelta('5 minutes')
    df = df[np.abs(df['time'] - single_time) <= time_threshold]

    return df

urls = [
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=ACB&station=ADG&station=AMN&station=ANJ&station=APN&station=ARB&station=AZO&station=BAX&station=BEH&station=BFA&station=BIV&station=BTL&station=CAD&station=CFS&station=CIU&station=CMX&station=CVX&station=D95&station=DET&station=DRM&station=DTW&station=DUH&station=ERY&station=ESC&station=FFX&station=FKS&station=FNT&station=FPK&station=GLR&station=GOV&station=GRR&station=HAI&station=HTL&station=HYX&station=IKW&station=IMT&station=IRS&station=ISQ&station=IWD&station=JXN&station=JYM&station=LAN&station=LDM&station=LWA&station=MBL&station=MBS&station=MCD&station=MGN&station=MKG&station=MNM&station=MOP&station=MTC&station=OEB&station=ONZ&station=OSC&station=OZW&station=P53&station=P58&station=P59&station=P75&station=PHN&station=PLN&station=PTK&station=PZQ&station=RMY&station=RNP&station=RQB&station=SAW&station=SJX&station=SLH&station=TEW&station=TTF&station=TVC&station=VLL&station=Y31&station=Y70&station=YIP&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4',
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=2P2&station=3D2&station=4R5&station=57C&station=82C&station=AIG&station=ARV&station=ASX&station=ATW&station=AUW&station=BCK&station=BUU&station=C29&station=C35&station=CLI&station=CMY&station=CWA&station=D25&station=DLL&station=EAU&station=EFT&station=EGV&station=ENW&station=ETB&station=EZS&station=FLD&station=GRB&station=HYR&station=ISW&station=JVL&station=LNL&station=LNR&station=LSE&station=LUM&station=MDZ&station=MFI&station=MKE&station=MRJ&station=MSN&station=MTW&station=MWC&station=OCQ&station=OEO&station=OLG&station=OSH&station=OVS&station=PBH&station=PCZ&station=PDC&station=PVB&station=RAC&station=RCX&station=RHI&station=RNH&station=RPD&station=RRL&station=RYV&station=RZN&station=SBM&station=STE&station=SUE&station=SUW&station=TKV&station=UBE&station=UES&station=UNU&station=VOK&station=Y23&station=Y50&station=Y51&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4',
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=04W&station=14Y&station=21D&station=3N8&station=9MN&station=ACQ&station=ADC&station=AEL&station=AIT&station=ANE&station=AQP&station=AUM&station=AXN&station=BBB&station=BDE&station=BDH&station=BFW&station=BJI&station=BRD&station=CBG&station=CDD&station=CFE&station=CKC&station=CKN&station=CNB&station=COQ&station=CQM&station=D39&station=DLH&station=DTL&station=DVP&station=DXX&station=DYT&station=ELO&station=ETH&station=EVM&station=FBL&station=FCM&station=FFM&station=FGN&station=FKA&station=FOZ&station=FRM&station=FSE&station=GDB&station=GHW&station=GNA&station=GPZ&station=GYL&station=HCD&station=HCO&station=HIB&station=HZX&station=ILL&station=INL&station=JKJ&station=JMR&station=JYG&station=LJF&station=LVN&station=LXL&station=LYV&station=MGG&station=MIC&station=MJQ&station=MKT&station=MML&station=MOX&station=MSP&station=MVE&station=MWM&station=MZH&station=ONA&station=ORB&station=OTG&station=OVL&station=OWA&station=P61&station=PEX&station=PKD&station=PNM&station=PQN&station=PWC&station=RGK&station=ROS&station=ROX&station=RRT&station=RST&station=RWF&station=RYM&station=SAZ&station=SGS&station=STC&station=STP&station=SYN&station=TKC&station=TOB&station=TVF&station=TWM&station=ULM&station=VVV&station=VWU&station=XVG&station=Y49&station=Y63&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4',
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=ADU&station=AIO&station=ALO&station=AMW&station=AWG&station=AXA&station=BNW&station=BRL&station=CAV&station=CBF&station=CCY&station=CID&station=CIN&station=CKP&station=CNC&station=CSQ&station=CWI&station=DBQ&station=DEH&station=DNS&station=DSM&station=DVN&station=EBS&station=EOK&station=EST&station=FFL&station=FOD&station=FSW&station=FXY&station=GGI&station=HNR&station=HPT&station=I75&station=ICL&station=IFA&station=IIB&station=IKV&station=IOW&station=LRJ&station=LWD&station=MCW&station=MIW&station=MPZ&station=MUT&station=MXO&station=OLZ&station=OOA&station=ORC&station=OTM&station=OXV&station=PEA&station=PRO&station=RDK&station=SDA&station=SHL&station=SLB&station=SPW&station=SUX&station=SXK&station=TNU&station=TVK&station=VTI&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4',
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=06C&station=1H2&station=3LF&station=AAA&station=AJG&station=ALN&station=ARR&station=BLV&station=BMI&station=C09&station=C75&station=CGX&station=CIR&station=CMI&station=CPS&station=CUL&station=DEC&station=DKB&station=DNV&station=DPA&station=ENL&station=FEP&station=FOA&station=FWC&station=GBG&station=HSB&station=I63&station=IGQ&station=IJX&station=IKK&station=JOT&station=LOT&station=LWV&station=M30&station=MDH&station=MDW&station=MLI&station=MQB&station=MTO&station=MVN&station=MWA&station=OLY&station=ORD&station=PIA&station=PNT&station=PPQ&station=PRG&station=PWK&station=RFD&station=RPJ&station=RSV&station=SAR&station=SFY&station=SLO&station=SPI&station=SQI&station=TAZ&station=TIP&station=UGN&station=UIN&station=VYS&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4',
    'https://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?station=1II&station=2R2&station=4I7&station=AID&station=ANQ&station=ASW&station=BAK&station=BFR&station=BMG&station=C62&station=C65&station=CFJ&station=DCY&station=EKM&station=EVV&station=EYE&station=FKR&station=FRH&station=FWA&station=GEZ&station=GGP&station=GPC&station=GSH&station=GUS&station=GWB&station=GYY&station=HBE&station=HFY&station=HHG&station=HLB&station=HNB&station=HUF&station=IMS&station=IND&station=JVY&station=LAF&station=MCX&station=MGC&station=MIE&station=MQJ&station=MZZ&station=OKK&station=OXI&station=PLD&station=PPO&station=RCR&station=RID&station=RZL&station=SBN&station=TYQ&station=UMP&station=UWL&station=VPZ&data=all&year1=2019&month1=7&day1=19&year2=2019&month2=7&day2=21&tz=Etc%2FUTC&format=onlycomma&latlon=yes&elev=no&missing=M&trace=T&direct=no&report_type=1&report_type=3&report_type=4']

In [ ]:
directory = 'C:\\Users\\Tony\\Desktop\\METR 4990\\new'
for filename in os.listdir(directory):
    if filename.endswith(".nc"):
        file_path = os.path.join(directory, filename)
        ds = read_file(file_path)
        ds = filter(ds)
        ds = remove_clutter(ds, skip_weak_ll_echo=True)
        df = read_multiple_urls(urls)
        df = preprocess_data(df)
        df = obs(df, ds)
        plot_image(df, ds)